# Lab: Using LangChain with IBM WatsonX

## 1. Intro to LangChain

[LangChain](https://docs.langchain.com/docs/) is an open-source development framework designed to simplify the creation of applications using large language models (LLMs).

The core idea of the library is that we can "chain" together different components to create more advanced use cases around LLMs. Here are the main components for the LangChain

- Model: interact with various LLMs
- Prompts: text that is sent to the LLMs
- Chains: allow to combine different LLM calls and actions automatically
- Embeddings and Vector Stores: break large data into chunks and store those to be queried when relevant
- Agents: enbale the LLMs to dynamically decide which tools to use in order to best respond to a given query

In short, **Langchain is a framework that can orchestrate a series of prompts to achieve a desired outcomes.**


## 2. How to connect LangChain to WatsonX.ai

In [1]:
!pip install chromadb==0.4.2 \
ibm-watson-machine-learning==1.0.311 \
ipywidgets==8.0.7 \
jupyter==1.0.0 \
langchain==0.0.236 \
matplotlib==3.7.2 \
numpy==1.24.2 \
pandas==1.5.3 \
plotly==5.15.0 \
pypdf==3.12.2 \
python-dotenv==1.0.0 \
requests==2.31.0 \
urllib3==1.26.11 \
rouge==1.0.1 \
scikit-learn==1.2.2 \
sentence-transformers==2.2.2 \
streamlit==1.24.1 \
safetensors==0.3.1  \
PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 93.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.9/139.9 kB 35.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 115.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 23.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 95.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 47.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 38.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 18.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━

In [25]:
!pip install chromadb

In [2]:
import os
os.environ['IBM_CLOUD_API_KEY'] = ''
os.environ['WATSONX_AI_ENDPOINT'] = 'https://us-south.ml.cloud.ibm.com'
os.environ['PROJECT_ID'] = '156312e8-6d77-41b3-978f-efa1aed6e1a1'

In [3]:
import os
from dotenv import load_dotenv
from typing import Any, List, Mapping, Optional, Union, Dict
from pydantic import BaseModel, Extra
try:
    from langchain import PromptTemplate
    from langchain.chains import LLMChain, SimpleSequentialChain
    from langchain.document_loaders import PyPDFLoader
    from langchain.indexes import VectorstoreIndexCreator #vectorize db index with chromadb
    from langchain.embeddings import HuggingFaceEmbeddings #for using HugginFace embedding models
    from langchain.text_splitter import CharacterTextSplitter #text splitter
    from langchain.llms.base import LLM
    from langchain.llms.utils import enforce_stop_tokens
except ImportError:
    raise ImportError("Could not import langchain: Please install ibm-generative-ai[langchain] extension.")

from ibm_watson_machine_learning.foundation_models import Model
from ibm_watson_machine_learning.metanames import GenTextParamsMetaNames as GenParams

In [4]:
#config Watsonx.ai environment
load_dotenv()
api_key = os.getenv("IBM_CLOUD_API_KEY", None)
ibm_cloud_url = os.getenv("WATSONX_AI_ENDPOINT", None)
project_id = os.getenv("PROJECT_ID", None)
if api_key is None or ibm_cloud_url is None or project_id is None:
    print("Ensure you copied the .env file that you created earlier into the same directory as this notebook")
else:
    creds = {
        "url": ibm_cloud_url,
        "apikey": api_key 
    }

In [5]:
##initializing WatsonX model
params = {
    GenParams.DECODING_METHOD: "sample",
    GenParams.MIN_NEW_TOKENS: 1,
    GenParams.MAX_NEW_TOKENS: 100,
    GenParams.RANDOM_SEED: 42,
    GenParams.TEMPERATURE: 0.5,
    GenParams.TOP_K: 50,
    GenParams.TOP_P:1
}

model = Model(
    model_id='google/flan-ul2',
    params=params,
    credentials=creds,
    project_id=project_id)


In order to use WatsonX-based LLMs with Langchain, the LLM object must be of class `BaseLanguageModel` (see [Langchain docs](https://api.python.langchain.com/en/latest/schema/langchain.schema.language_model.BaseLanguageModel.html)). We'll use the custom class below to accomplish this.

In [6]:
# Wrap the WatsonX Model in a langchain.llms.base.LLM subclass to allow LangChain to interact with the model

class LangChainInterface(LLM, BaseModel):
    credentials: Optional[Dict] = None
    model: Optional[str] = None
    params: Optional[Dict] = None
    project_id : Optional[str]=None

    class Config:
        """Configuration for this pydantic object."""
        extra = Extra.forbid

    @property
    def _identifying_params(self) -> Mapping[str, Any]:
        """Get the identifying parameters."""
        _params = self.params or {}
        return {
            **{"model": self.model},
            **{"params": _params},
        }
    
    @property
    def _llm_type(self) -> str:
        """Return type of llm."""
        return "IBM WATSONX"

    def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:
        """Call the WatsonX model"""
        params = self.params or {}
        model = Model(model_id=self.model, params=params, credentials=self.credentials, project_id=self.project_id)
        text = model.generate_text(prompt)
        if stop is not None:
            text = enforce_stop_tokens(text, stop)
        return text

llm_model = LangChainInterface(model='google/flan-ul2', credentials=creds, params=params, project_id=project_id)

In [7]:
##predict with the model
text = "Where is the capital of South Korea"
llm_model(text)

'seoul'

## 3. Prompt Templates & Chains

In the previous example, the user input is sent directly to the LLM. However, when using an LLM in an application, you will usually need to reuse the same prompt across multiple scenarios

- Accepting user input and contruct a prompt
- Generating mutiple prompts from an collection of data points in a dataset 

In [8]:
# Define the prompt templates
prompt = PromptTemplate(
  input_variables=["country"],
  template= "where is the capital of {country}?",
)

# Chaining 
chain = LLMChain(llm=llm_model, prompt=prompt)

# Getting predictions
countries = ["USA", "England", "Japan", "Saudi Arabia"]
for country in countries:
    response = chain.run(country)
    print(prompt.format(country=country) + " = " + response)

where is the capital of USA? = washington dc
where is the capital of England? = london
where is the capital of Japan? = tokyo
where is the capital of Saudi Arabia? = jeddah


## 4. Simple sequential chains
The utility of LangChain becomes apparent as we chain outputs of one model as input to another model. Here's a simple example where one generates a question which the other model answers.

LangChain determines a model's output based on its response.  In our examples, the first model creates a response to the end prompt of "Question:" which LangChain maps as an input variable called "question" which it passes to the 2nd model.

In [9]:
## Create two sequential prompts 
pt1 = PromptTemplate(input_variables=["topic"], template="Generate a random question about {topic}: Question: ")
pt2 = PromptTemplate(
    input_variables=["question"],
    template="Answer the following question: {question}",
)

In [10]:
flan = LangChainInterface(model='google/flan-ul2', credentials=creds, params=params, project_id=project_id)
model = LangChainInterface(model='google/flan-ul2', credentials=creds, project_id=project_id)

In [11]:
prompt_to_flan = LLMChain(llm=flan, prompt=pt1)
flan_to_model = LLMChain(llm=model, prompt=pt2)
qa = SimpleSequentialChain(chains=[prompt_to_flan, flan_to_model], verbose=True)

In [12]:
qa.run("artificial intelligence")



> Entering new SimpleSequentialChain chain...
What does the term "artificial intelligence" mean?
intelligent computer program

> Finished chain.


'intelligent computer program'

## 5. Easy Loading of Documents Using Lang Chain
LangChain makes it easy to extract passages from documents so that you can answering questions based on your document's content.

In [23]:
!wget https://github.com/hwchase17/chroma-langchain/blob/master/state_of_the_union.txt -O state_of_the_union.txt

--2023-11-20 07:00:33--  https://github.com/hwchase17/chroma-langchain/blob/master/state_of_the_union.txt
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 47228 (46K) [text/plain]
Saving to: ‘state_of_the_union.txt’

state_of_the_union. 100%[===================>]  46.12K  --.-KB/s    in 0.06s   

2023-11-20 07:00:33 (734 KB/s) - ‘state_of_the_union.txt’ saved [47228/47228]



In [24]:
!ls

state_of_the_union.txt	what_is_generative_ai.pdf


In [33]:
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader

In [27]:
loader = TextLoader('state_of_the_union.txt')
documents = loader.load()

In [28]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

In [29]:
embeddings = HuggingFaceEmbeddings()
vectordb = Chroma.from_documents(texts, embeddings)

In [38]:
###initializing watsonx flan_ul2 model
params = {
    GenParams.DECODING_METHOD: "sample",
    GenParams.MIN_NEW_TOKENS: 50,
    GenParams.MAX_NEW_TOKENS: 300,
    GenParams.TEMPERATURE: 0.2,
    GenParams.TOP_K: 100,
    GenParams.TOP_P:1
}

model = LangChainInterface(model='google/flan-ul2', credentials=creds, params=params, project_id=project_id)
model_llama = LangChainInterface(model='meta-llama/llama-2-70b-chat', credentials=creds, params=params, project_id=project_id)

In [39]:
qa = RetrievalQA.from_chain_type(llm=model, chain_type="stuff", retriever=vectordb.as_retriever())

In [40]:
query = "What did the president say about Ketanji Brown Jackson"
qa.run(query)

'One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence. ","","A former top litigator in private practice. A former federal public defender. And from a family of public school educators and police officers.'

In [44]:
chain = RetrievalQA.from_chain_type(llm=model_llama, 
                                    chain_type="stuff", 
                                    retriever=vectordb.as_retriever(), 
                                    input_key="question")

In [45]:
##answering based on the documents 
chain.run("What did the president say about Ketanji Brown Jackson?")

" The president said that he nominated Ketanji Brown Jackson to the Supreme Court 4 days ago, and that she is one of the nation's top legal minds and will continue Justice Breyer's legacy of excellence. She is a former top litigator in private practice, a former federal public defender, and from a family of public school educators and police officers."